# Chapter 4

# Camera Models and Augmented Reality

## 4.1 The Pin-hole Camera Model 


### pin-hole camera model 참고

핀홀 카메라 모델은 아래 그림과 같이 하나의 바늘구멍(pinhole)을 통해 외부의 상이 이미지로 투영된다는 모델. 
이 바늘구멍(pinhole)이 렌즈 중심에 해당되며 이곳에서 뒷면의 상이 맺히는 곳까지의 거리가 카메라 초점거리 임.


![](pinhole.jpg)
<그림 > pinhole 카메라 모델 (그림출처: 위키피디아)


 
### 카메라 초점거리(focal length)는 렌즈에서 이미지 센서까지의 거리
![](http://cfile6.uf.tistory.com/image/2535F04A52672EB427032B)
<그림 > 렌즈-이미지센서 투영


초점거리라 하면 볼록렌즈의 초점을 생각하기 쉬운데, 여기서(카메라 모델) 말하는 초점거리는 렌즈중심과 이미지센서(CCD, CMOS 등)와의 거리를 말함.

디지털 카메라 등에서 초점거리는 mm 단위로 표현되지만 카메라 모델에서 말하는 카메라 초점거리는 픽셀(pixel) 단위로 표현됩니다. 즉, fx, fy의 단위가 픽셀임.
 

초점으로부터 거리가 1(unit distance)인 평면을 normalized image plane이라고 부르며 이 평면상의 좌표를 보통 normalized image coordinate라고 부릅니다. 물론 이것은 실제는 존재하지 않는 가상의(상상의) 이미지 평면입니다. 카메라 좌표계 상의 한 점 (Xc, Yc, Zc)를 영상좌표계로 변환할 때 먼저 Xc, Yc를 Zc(카메라 초점에서의 거리)로 나누는 것은 이 normalized image plane 상의 좌표로 변환하는 것이며, 여기에 다시 초점거리 f를 곱하면 우리가 원하는 실제 영상좌표(pixel)가 나옵니다 (아래 그림 참조).

![](http://cfile29.uf.tistory.com/image/017ADB3A511435821DF008)



[출처 : http://darkpgmr.tistory.com/  다크 프로그래머]

### The pin-hole camera model
![](4-1.jpg)

Figure 4.1: The pin-hole camera model. The image point x is at the intersection of the
image plane and the line joining the 3D point X and the camera center C. The dashed
line is the optical axis of the camera.



## Projecting 3D Points
### Let’s create a camera class to handle all the operations we need for modeling cameras and projections:

In [5]:
from numpy import *
from scipy import linalg 

class Camera(object):
    """ Class for representing pin-hole cameras. """

    def __init__(self,P):
        """ Initialize P = K[R|t] camera model. """
        self.P = P
        self.K = None # calibration matrix self.R = None # rotation
        self.t = None # translation
        self.c = None # camera center

    def project(self,X):
        """ Project points in X (4*n array) and normalize coordinates. """
        x = dot(self.P,X) 
        for i in range(3):
            x[i] /= x[2] 
        return x

### The example below shows how to project 3D points into an image view. 
### In this example, we will use one of  the Oxford multi-view datasets, the “Model House” data set, available at  http://www.robots.ox.ac.uk/~vgg/data/data-mview.html. Download the 3D geometry file and copy the house.p3d file to your working directory:

In [7]:

import camera

# load points
points = loadtxt('house.p3d').T
points = vstack((points,ones(points.shape[1])))

# setup camera
P = hstack((eye(3),array([[0],[0],[-10]]))) 
cam = camera.Camera(P)
x = cam.project(points)

# plot projection
figure() 
plot(x[0],x[1],'k.') 
show()

ImportError: No module named camera

### To see how moving the camera changes the projection, try the following piece of code that incrementally rotates the camera around a random 3D axis:

In [8]:
# create transformation
r = 0.05*random.rand(3)
rot = camera.rotation_matrix(r)
# rotate camera and project
figure()
for t in range(20):
    cam.P = dot(cam.P,rot)
    x = cam.project(points)
plot(x[0],x[1],’k.’)
show()

SyntaxError: invalid syntax (<ipython-input-8-d19050abacdf>, line 9)

### Here we used the helper function rotation_matrix(), which creates a rotation matrix
### for 3D rotations around a vector (add this to camera.py):

In [8]:
def rotation_matrix(a):
    """ Creates a 3D rotation matrix for rotation around the axis of the vector a. """
    R = eye(4)
    R[:3,:3] = linalg.expm([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]]) 
    return R

![](4-2.jpg)

Figure 4.2: An example of projecting 3D points. (left) sample image. (middle) projected
points into a view. (right) trajectory of projected points under camera rotation.
Data from the Oxford "Model House" dataset

## Factoring the Camera Matrix
### If we are given a camera matrix P of the form in equation (4.2), we need to be able to recover the internal parameters K and the camera position and pose t and R.
### Partitioning the matrix is called factorization. In this case, we will use a type of matrix factorization called RQ-factorization.
### Add the following method to the Camera class:

In [9]:
def factor(self):
    """ Factorize the camera matrix into K,R,t as P = K[R|t]. """
    # factor first 3*3 part
    K,R = linalg.rq(self.P[:,:3])
    # make diagonal of K positive
    T = diag(sign(diag(K)))
    if linalg.det(T) < 0:
        T[1,1] *= -1
    self.K = dot(K,T)
    self.R = dot(T,R) # T is its own inverse
    self.t = dot(linalg.inv(self.K),self.P[:,3])
    return self.K, self.R, self.t

### RQ-factorization is not unique, there is a sign ambiguity in the factorization. Since we need the rotation matrix R to have positive determinant (otherwise the coordinate axis can get flipped), we can add a transform T to change the sign when needed.
### Try this on a sample camera to see that it works:


In [10]:
import camera

K = array([[1000,0,500],[0,1000,300],[0,0,1]])
tmp = camera.rotation_matrix([0,0,1])[:3,:3]
Rt = hstack((tmp,array([[50],[40],[30]])))
cam = camera.Camera(dot(K,Rt))

print K,Rt
print cam.factor()

ImportError: No module named camera

## Computing the Camera Center
### Given a camera projection matrix, P, it is useful to be able to compute the camera’s position in space. The camera center, C, is a 3D point with the property P C = 0. For acamerawith  P =K [R|t], this gives
## K[R | t]C = KRC + Kt = 0, 
### and the camera center can be computed as
## C = −RT t.

In [14]:
def center(self):

    if self.c is not None:
        return self.c
    else:
    # compute c by factoring
    self.factor()
    self.c = -dot(self.R.T,self.t)
    return self.c

IndentationError: expected an indented block (<ipython-input-14-9194a5323978>, line 7)

## 4.2 Camera Calibration


### 카메라 캘리브레이션(camera calibration)은 위와 같은 3D 공간좌표와 2D 영상좌표 사이의 변환관계 또는 이 변환관계를 설명하는 파라미터를 찾는 과정

### 핀홀(pinhole) 카메라 모델에서 이러한 변환 관계는 다음과 같이 모델링 됨.

![](matrix.jpg)

여기서, (X,Y,Z)는 월드 좌표계(world coordinate system) 상의 3D 점의 좌표, [R|t]는 월드 좌표계를 카메라 좌표계로 변환시키기 위한 회전/이동변환 행렬이며 A는 camera matrix

![](calibration.jpg)
 
 
 식(1)에서 [R|t]를 카메라 외부 파라미터(extrinsic parameter), A를 내부 파라미터(intrinsic parameter)라고 함.
 
 카메라 외부 파라미터는 카메라의 설치 높이, 방향(팬, 틸트) 등 카메라와 외부 공간과의 기하학적 관계에 관련된 파라미터이며 내부 파라미터는 카메라의 초점 거리, aspect ratio, 중심점 등 카메라 자체의 내부적인 파라미터를 의미
 
 ![](4-3.jpg)
Figure 4.3: A simple camera calibration setup. (left) an image of the setup used.
(right) the image used for the calibration. Measuring the width and height of the
calibration object in the image and the physical dimensions of the setup is enough to
determine the focal length.

## 4.3 Pose Estimation from Planes and Markers

### we saw how to estimate homographies between planes. 
### Combining this with a calibrated camera makes it possible to compute the camera’s pose
### Let’s illustrate with an example. Consider the two top images in Figure 4-4. The follow- ing code will extract SIFT features in both images and robustly estimate a homography using RANSAC:

![](4-4.jpg)
Figure 4.4: Example of computing the projection matrix for a new view using a planar
object as marker. Matching image features to an aligned marker gives a homography
that can be used to compute the pose of the camera. (top left) template image with
a blue square. (top right) an image taken from an unknown viewpoint with the same
square transformed with the estimated homography. (bottom) a cube transformed
using the estimated camera matrix.




In [17]:
import homography 
import camera 
import sift
# compute features
sift.process_image('book_frontal.JPG','im0.sift') 
l0,d0 = sift.read_features_from_file('im0.sift')
sift.process_image('book_perspective.JPG','im1.sift') 
l1,d1 = sift.read_features_from_file('im1.sift')

ImportError: No module named homography

In [20]:
# match features and estimate homography
matches = sift.match_twosided(d0,d1)
ndx = matches.nonzero()[0]
fp = homography.make_homog(l0[ndx,:2].T) 
ndx2 = [int(matches[i]) for i in ndx]
tp = homography.make_homog(l1[ndx2,:2].T)

model = homography.RansacModel()
H = homography.H_from_ransac(fp,tp,model)

NameError: name 'sift' is not defined

### To check our results, we will need some simple 3D object placed on the marker. Here we will use a cube and generate the cube points using the function:


In [22]:
def cube_points(c,wid):
    p = []
    # bottom
    p.append([c[0]-wid,c[1]-wid,c[2]-wid]) 
    p.append([c[0]-wid,c[1]+wid,c[2]-wid]) 
    p.append([c[0]+wid,c[1]+wid,c[2]-wid]) 
    p.append([c[0]+wid,c[1]-wid,c[2]-wid]) 
    p.append([c[0]-wid,c[1]-wid,c[2]-wid])
    # top
    p.append([c[0]-wid,c[1]-wid,c[2]+wid]) 
    p.append([c[0]-wid,c[1]+wid,c[2]+wid]) 
    p.append([c[0]+wid,c[1]+wid,c[2]+wid]) 
    p.append([c[0]+wid,c[1]-wid,c[2]+wid]) 
    p.append([c[0]-wid,c[1]-wid,c[2]+wid])
    # vertical sides
    p.append([c[0]-wid,c[1]-wid,c[2]+wid]) 
    p.append([c[0]-wid,c[1]+wid,c[2]+wid]) 
    p.append([c[0]-wid,c[1]+wid,c[2]-wid]) 
    p.append([c[0]+wid,c[1]+wid,c[2]-wid]) 
    p.append([c[0]+wid,c[1]+wid,c[2]+wid]) 
    p.append([c[0]+wid,c[1]-wid,c[2]+wid]) 
    p.append([c[0]+wid,c[1]-wid,c[2]-wid])
    # same as first to close plot
    # same as first to close plot
    return array(p).T

### Some points are reoccurring so that plot() will generate a nice-looking cube.
### With a homography and a camera calibration matrix, we can now determine the relative transformation between the two views:


In [25]:
# camera calibration
K = my_calibration((747,1000))

# 3D points at plane z=0 with sides of length 0.2
box = cube_points([0,0,0.1],0.1)

# project bottom square in first image
cam1 = camera.Camera( hstack((K,dot(K,array([[0],[0],[-1]])) )) )

# first points are the bottom square
box_cam1 = cam1.project(homography.make_homog(box[:,:5]))

# use H to transfer points to the second image
box_trans = homography.normalize(dot(H,box_cam1))

# compute second camera matrix from cam1 and H
cam2 = camera.Camera(dot(H,cam1.P))
A = dot(linalg.inv(K),cam2.P[:,:3])
A = array([A[:,0],A[:,1],cross(A[:,0],A[:,1])]).T 
cam2.P[:,:3] = dot(K,A)

# project with the second camera
box_cam2 = cam2.project(homography.make_homog(box))

# test: projecting point on z=0 should give the same
point = array([1,1,0,1]).T
print homography.normalize(dot(dot(H,cam1.P),point)) 
print cam2.project(point)


NameError: name 'my_calibration' is not defined

## 4.4 Augmented Reality

### Augmented reality (AR) is a collective term for placing objects and information on top of image data. The classic example is placing a 3D computer graphics model so that it looks like it belongs in the scene, and moves naturally with the camera motion in the case of video.

## From Camera Matrix to OpenGL Format
### OpenGL uses 4 × 4 matrices to represent transforms (both 3D transforms and projec- tions). This is only slightly different from our use of 3 × 4 camera matrices.

### Given that we have a camera calibrated so that the calibration matrix K is known, the following function translates the camera properties to an OpenGL projection matrix:


In [28]:
def set_projection_from_camera(K):
    """ Set view from a camera calibration matrix. """
    glMatrixMode(GL_PROJECTION) 
    glLoadIdentity()
    fx = K[0,0]
    fy = K[1,1]
    fovy = 2*arctan(0.5*height/fy)*180/pi 
    aspect = (width*fy)/(height*fx)
    
    # define the near and far clipping planes
    near = 0.1 
    far = 100.0
    
    # set perspective
    gluPerspective(fovy,aspect,near,far) 
    glViewport(0,0,width,height)

In [30]:
def set_modelview_from_camera(Rt):
    """ Set the model view matrix from camera pose. """
    glMatrixMode(GL_MODELVIEW) 
    glLoadIdentity()
    
    # rotate teapot 90 deg around x-axis so that z-axis is up
    Rx = array([[1,0,0],[0,0,-1],[0,1,0]])
    
    # set rotation to best approximation
    R = Rt[:,:3]
    U,S,V = linalg.svd(R)
    R = dot(U,V)
    R[0,:] = -R[0,:] # change sign of x-axis
    
    # set translation
    t = Rt[:,3]
    
    # setup 4*4 model view matrix
    M = eye(4)
    M[:3,:3] = dot(R,Rx) 
    M[:3,3] = t
    
    # transpose and flatten to get column order
    M = M.T
    m = M.flatten()
    # replace model view with the new matrix
    glLoadMatrixf(m)


## Placing Virtual Objects in the Image
### The first thing we need to do is to add the image (the one we want to place virtual objects in) as a background. In OpenGL this is done by creating a quadrilateral, a quad, that fills the whole view. 

### This function loads an image, converts it to an OpenGL texture, and places that texture on the quad:

In [31]:
def draw_background(imname):
    """ Draw background image using a quad. """
    # load background image (should be .bmp) to OpenGL texture
    bg_image = pygame.image.load(imname).convert() 
    bg_data = pygame.image.tostring(bg_image,"RGBX",1)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # bind the texture
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D,glGenTextures(1)) 
    glTexImage2D(GL_TEXTURE_2D,0,GL_RGBA,width,height,0,GL_RGBA,GL_UNSIGNED_BYTE,bg_data) 
    glTexParameterf(GL_TEXTURE_2D,GL_TEXTURE_MAG_FILTER,GL_NEAREST) 
    glTexParameterf(GL_TEXTURE_2D,GL_TEXTURE_MIN_FILTER,GL_NEAREST)
    
    # create quad to fill the whole window
    glBegin(GL_QUADS)
    glTexCoord2f(0.0,0.0); glVertex3f(-1.0,-1.0,-1.0) 
    glTexCoord2f(1.0,0.0); glVertex3f( 1.0,-1.0,-1.0) 
    glTexCoord2f(1.0,1.0); glVertex3f( 1.0, 1.0,-1.0) 
    glTexCoord2f(0.0,1.0); glVertex3f(-1.0, 1.0,-1.0) 
    glEnd()
    # clear the texture
    glDeleteTextures(1)

## Tying It All Together
### The full script for generating an image like the one in Figure 4-5 looks like this (assuming that you also have the functions introduced above in the same file):

In [32]:
from OpenGL.GL import * 
from OpenGL.GLU import * 
from OpenGL.GLUT import * 
import pygame, pygame.image 
from pygame.locals import * 
import pickle

width,height = 1000,747

def setup():
    """ Setup window and pygame environment. """
    pygame.init() 
    pygame.display.set_mode((width,height),OPENGL | DOUBLEBUF) 
    pygame.display.set_caption('OpenGL AR demo')
    
    # load camera data
    with open('ar_camera.pkl','r') as f: 
        K = pickle.load(f)
        Rt = pickle.load(f)
    setup() 
    draw_background('book_perspective.bmp') 
    set_projection_from_camera(K) 
    set_modelview_from_camera(Rt) 
    draw_teapot(0.02)
    
    while True:
        event = pygame.event.poll()
        if event.type in (QUIT,KEYDOWN):
            break 
    pygame.display.flip()


ImportError: No module named OpenGL.GL

![](4-6.jpg)
Figure 4.6: Loading a 3D model from an .obj file and placing it on a book in a scene
using camera parameters computed from feature matches.